# Can we find unique clients in data?
Transactions in data looks like are independent of each other. Perhaps the organizers made it for better data anonymization. But what if we find transactions belonging to the same user? May be it will help someone in this competition.

I accidentally saw some magic in the feature V307

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from scipy.stats.stats import pearsonr
import hashlib
import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100

import os
print(os.listdir("../input/features/"))

['train_y.pkl', 'train_uid_features.pkl', 'train_features.pkl', 'useful_features.npy', 'test_features.pkl', 'test_uid_features.pkl', 'train_basic_features.pkl', 'rm_features.npy', 'test.pkl', 'test_basic_features.pkl', 'train.pkl']


In [2]:
train = pd.read_pickle('../input/features/train.pkl')
test = pd.read_pickle('../input/features/test.pkl')
train_y = train['isFraud'].copy()

train= train.reset_index()
test = test.reset_index()

print(train.shape)
print(train_y.shape)
print(test.shape)

(590540, 434)
(590540,)
(506691, 434)


In [3]:
train['card1'] = train['card1'].fillna(0)
train['card2'] = train['card2'].fillna(0)
train['card3'] = train['card3'].fillna(0)
train['card5'] = train['card5'].fillna(0)
train['card4'] = train['card4'].fillna('nan')
train['card6'] = train['card6'].fillna('nan')

### Functions that will help us find unique devices and unique cards. (I think it is clear for you why we use hash functions)

In [4]:
def card_info_hash(x):
    s = (str(int(x['card1']))+
         str(int(x['card2']))+
         str(int(x['card3']))+
         str(x['card4'])+
         str(int(x['card5']))+
         str(x['card6']))
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h

In [5]:
def device_hash(x):
    s =  str(x['id_30'])+str(x['id_31'])+str(x['id_32'])+str(x['id_33'])+str( x['DeviceType'])+ str(x['DeviceInfo'])
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h


In [6]:
train['card_hash'] = train.apply(lambda x: card_info_hash(x), axis=1   )
train['device_hash'] = train.apply(lambda x: device_hash(x), axis=1   )

In [7]:
def get_data_by_card_hash( data, card_hash):
    mask = data['card_hash']==card_hash
    return data.loc[mask,:].copy()


def get_data_by_device_hash( data, device_hash):
    mask = data['device_hash']==device_hash
    return data.loc[mask,:].copy()


def get_data_by_card_and_device_hash( data, card_hash, device_hash):
    mask = (data['card_hash']==card_hash) &(data['device_hash']==device_hash)
    return data.loc[mask,:].copy()

In [8]:
s = train.groupby(['card_hash' , 'device_hash'])['isFraud'].agg(['mean', 'count'])

In [9]:
s[(s['mean']==1) & (s['count']>15) ].head(500)

mean  count
card_hash       device_hash                 
42c398e0ff9f923 6b24aeb6313e820   1.0     24
620227200c64fb9 4254e5002205af0   1.0     17
                abda5e39116a69d   1.0     21
7086d86140f07a4 c7f2e00e03ae096   1.0     46
72b54c868f8ba62 84edf52bf2da6b1   1.0     16
751777fefa9891d 669f9256b04fb02   1.0     41
a2bcb561fce2b69 57839ffc2020545   1.0     21
b00a5853e414876 0f8eda5b168c6aa   1.0     38
                11b06c5567f6dc8   1.0     17
b4f15ed9e7c1e0a b62aa9813bf1ea8   1.0     18
f8360028f0c401d 4682f7ab67f956b   1.0    112
                8c77e8689f54c7e   1.0     57

In [10]:
very_strange_thing = get_data_by_card_and_device_hash(train, '751777fefa9891d', '669f9256b04fb02')

In [11]:
very_strange_thing[[ 'TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'device_hash','card_hash', 'V307']]

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,device_hash,card_hash,V307
535581,3522581,1,14094661,25.453125,C,669f9256b04fb02,751777fefa9891d,0.000000
537276,3524276,1,14157389,26.500000,C,669f9256b04fb02,751777fefa9891d,25.452700
537444,3524444,1,14160433,38.125000,C,669f9256b04fb02,751777fefa9891d,51.950600
537448,3524448,1,14160509,38.125000,C,669f9256b04fb02,751777fefa9891d,90.068199
537528,3524528,1,14161938,21.890625,C,669f9256b04fb02,751777fefa9891d,128.185806
537541,3524541,1,14162075,21.890625,C,669f9256b04fb02,751777fefa9891d,150.072693
537545,3524545,1,14162137,21.890625,C,669f9256b04fb02,751777fefa9891d,171.959595
537593,3524593,1,14163101,11.062500,C,669f9256b04fb02,751777fefa9891d,193.846497
537963,3524963,1,14170292,10.945312,C,669f9256b04fb02,751777fefa9891d,204.912903
537975,3524975,1,14170518,11.867188,C,669f9256b04fb02,751777fefa9891d,215.856293


# Don't see anything?

In [12]:
#magic
very_strange_thing['V307_diff'] = very_strange_thing['V307'].diff().shift(-1)

In [13]:
very_strange_thing['difference'] = very_strange_thing['V307_diff'] - very_strange_thing['TransactionAmt']

In [14]:
very_strange_thing[[ 'TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'device_hash','card_hash', 'V307', 'V307_diff', 'difference']]

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,device_hash,card_hash,V307,V307_diff,difference
535581,3522581,1,14094661,25.453125,C,669f9256b04fb02,751777fefa9891d,0.000000,25.452700,-0.000425
537276,3524276,1,14157389,26.500000,C,669f9256b04fb02,751777fefa9891d,25.452700,26.497900,-0.002100
537444,3524444,1,14160433,38.125000,C,669f9256b04fb02,751777fefa9891d,51.950600,38.117599,-0.007401
537448,3524448,1,14160509,38.125000,C,669f9256b04fb02,751777fefa9891d,90.068199,38.117607,-0.007393
537528,3524528,1,14161938,21.890625,C,669f9256b04fb02,751777fefa9891d,128.185806,21.886887,-0.003738
537541,3524541,1,14162075,21.890625,C,669f9256b04fb02,751777fefa9891d,150.072693,21.886902,-0.003723
537545,3524545,1,14162137,21.890625,C,669f9256b04fb02,751777fefa9891d,171.959595,21.886902,-0.003723
537593,3524593,1,14163101,11.062500,C,669f9256b04fb02,751777fefa9891d,193.846497,11.066406,0.003906
537963,3524963,1,14170292,10.945312,C,669f9256b04fb02,751777fefa9891d,204.912903,10.943390,-0.001923
537975,3524975,1,14170518,11.867188,C,669f9256b04fb02,751777fefa9891d,215.856293,11.865601,-0.001587


In [15]:
len(very_strange_thing)

41

> # Coincidence? i don't think so 

![](https://www.meme-arsenal.com/memes/06308418f56ad0d8d674c247e5ccba49.jpg )

### Highly likely V307 is a cumulative sum of transactions for a certain period for one unique client

This group of transactions has the same card hash and device hash, and 41 consecutively increasing numbers. Also all this transactions are fraud. 

May be this knowledge will help you to significantly improve your models. 

It doesn't work for all pairs of card and devices hashes, so you can improve alghoritm for searching unique clients.

# Bonus: You can try to decipher the values of some features

In [16]:
def dt_features(data):
    data = data.copy()
    
    start_dt = data['TransactionDT'].min()
    data['TransactionDT_norm']  = (data['TransactionDT'] - start_dt)/3600
    data['TransactionDT_norm_days'] = data['TransactionDT_norm']/24
    data['TransactionDT_diff'] = data['TransactionDT_norm'].diff().fillna(0)
    data['TransactionDT_diff_days'] = data['TransactionDT_diff']/24
    return data

In [17]:
very_strange_thing = dt_features(very_strange_thing)

In [18]:
list_of_interesting_features = [ 'TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'device_hash','card_hash', 'V307', 'V307_diff', 'difference', 
                                'TransactionDT_norm_days', 'TransactionDT_diff', 'TransactionDT_diff_days', 
                               
                               ] + ['D{}'.format(i) for i in range(1,16)]

In [20]:
very_strange_thing

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,...,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,card_hash,device_hash,V307_diff,difference,TransactionDT_norm,TransactionDT_norm_days,TransactionDT_diff,TransactionDT_diff_days
535581,3522581,1,14094661,25.453125,C,15885,545.0,185.0,visa,138.0,debit,NaN,NaN,NaN,NaN,gmail.com,gmail.com,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,M2,...,NaN,NaN,-20.0,404183.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,64.0,NaN,New,NotFound,225.0,NaN,266.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 52.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,CRO-L03 Build/HUAWEICRO-L03,751777fefa9891d,669f9256b04fb02,25.452700,-0.000425,0.000000,0.000000,0.000000,0.000000
537276,3524276,1,14157389,26.500000,C,15885,545.0,185.0,visa,138.0,debit,NaN,NaN,NaN,17.0,gmail.com,gmail.com,1.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.833496,0.833496,0.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,M2,...,NaN,NaN,-20.0,625177.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,64.0,NaN,Found,Found,225.0,NaN,266.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 52.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,CRO-L03 Build/HUAWEICRO-L03,751777fefa9891d,669f9256b04fb02,26.497900,-0.002100,17.424444,0.726019,17.424444,0.726019
537444,3524444,1,14160433,38.125000,C,15885,545.0,185.0,visa,138.0,debit,NaN,NaN,NaN,17.0,gmail.com,gmail.com,1.0,3.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,M2,...,NaN,NaN,-20.0,564558.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,64.0,NaN,Found,Found,225.0,NaN,266.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 52.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,CRO-L03 Build/HUAWEICRO-L03,751777fefa9891d,669f9256b04fb02,38.117599,-0.007401,18.270000,0.761250,0.845556,0.035231
537448,3524448,1,14160509,38.125000,C,15885,545.0,185.0,visa,138.0,debit,NaN,NaN,NaN,17.0,gmail.com,gmail.com,1.0,3.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,M2,...,NaN,NaN,-20.0,639473.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,64.0,NaN,Found,Found,225.0,NaN,266.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 52.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,CRO-L03 Build/HUAWEICRO-L03,751777fefa9891d,669f9256b04fb02,38.117607,-0.007393,18.291111,0.762130,0.021111,0.000880
537528,3524528,1,14161938,21.890625,C,15885,545.0,185.0,visa,138.0,debit,NaN,NaN,NaN,17.0,gmail.com,gmail.com,3.0,5.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,M2,...,NaN,NaN,-25.0,157538.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,64.0,NaN,Found,Found,225.0,NaN,266.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 52.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,CRO-L03 Build/HUAWEICRO-L03,751777fefa9891d,669f9256b04fb02,21.886887,-0.003738,18.688056,0.778669,0.396944,0.016539
537541,3524541,1,14162075,21.890625,C,15885,545.0,185.0,visa,138.0,debit,NaN,NaN,NaN,17.0,gmail.com,gmail.com,3.0,5.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,M2,...,NaN,NaN,-25.0,295042.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,64.0,NaN,Found,Found,225.0,NaN,266.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 52.0 for android,NaN,NaN,NaN,F,F,T,F,

In [19]:
very_strange_thing[list_of_interesting_features]

# look at  TransactionDT_norm_days and  D8	D9
# D1 looks like  TransactionDT_norm_days rounded to int

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,device_hash,card_hash,V307,V307_diff,difference,TransactionDT_norm_days,TransactionDT_diff,TransactionDT_diff_days,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
535581,3522581,1,14094661,25.453125,C,669f9256b04fb02,751777fefa9891d,0.000000,25.452700,-0.000425,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0
537276,3524276,1,14157389,26.500000,C,669f9256b04fb02,751777fefa9891d,25.452700,26.497900,-0.002100,0.726019,17.424444,0.726019,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.833496,0.833496,0.0,NaN,1.0,1.0,1.0,1.0
537444,3524444,1,14160433,38.125000,C,669f9256b04fb02,751777fefa9891d,51.950600,38.117599,-0.007401,0.761250,0.845556,0.035231,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0
537448,3524448,1,14160509,38.125000,C,669f9256b04fb02,751777fefa9891d,90.068199,38.117607,-0.007393,0.762130,0.021111,0.000880,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0
537528,3524528,1,14161938,21.890625,C,669f9256b04fb02,751777fefa9891d,128.185806,21.886887,-0.003738,0.778669,0.396944,0.016539,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0
537541,3524541,1,14162075,21.890625,C,669f9256b04fb02,751777fefa9891d,150.072693,21.886902,-0.003723,0.780255,0.038056,0.001586,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0
537545,3524545,1,14162137,21.890625,C,669f9256b04fb02,751777fefa9891d,171.959595,21.886902,-0.003723,0.780972,0.017222,0.000718,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.875000,0.875000,0.0,NaN,1.0,1.0,1.0,1.0
537593,3524593,1,14163101,11.062500,C,669f9256b04fb02,751777fefa9891d,193.846497,11.066406,0.003906,0.792130,0.267778,0.011157,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.916504,0.916504,0.0,NaN,1.0,1.0,1.0,1.0
537963,3524963,1,14170292,10.945312,C,669f9256b04fb02,751777fefa9891d,204.912903,10.943390,-0.001923,0.875359,1.997500,0.083229,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.000000,0.000000,0.0,NaN,1.0,1.0,1.0,1.0
537975,3524975,1,14170518,11.867188,C,669f9256b04fb02,751777fefa9891d,215.856293,11.865601,-0.001587,0.877975,0.062778,0.002616,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.000000,0.000000,0.0,NaN,1.0,1.0,1.0,1.0


In [ ]:
plt.figure(figsize=(14,6))
plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V202'] , label = 'V202' )
plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V203'] , label = 'V203')
plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V204'] , label = 'V204')

plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V306'] , label = 'V306')
plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V307'] , label = 'V307')
plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V308'] , label = 'V308')

plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V316'] , label = 'V316')
plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V317'] , label = 'V317')
plt.plot( very_strange_thing['TransactionDT_norm_days'], very_strange_thing['V318'] , label = 'V318')

plt.xlabel('Datetime')

plt.legend()
plt.show()

In [ ]:
list_of_interesting_features = [ 'TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'device_hash','card_hash', 'V307', 'V307_diff', 'difference', 
                                'TransactionDT_norm_days', 'TransactionDT_diff', 'TransactionDT_diff_days', 
                               
                               ] + ['C{}'.format(i) for i in range(1,15)]

In [ ]:
very_strange_thing[list_of_interesting_features]
# c2

![](https://www.dictionary.com/e/wp-content/uploads/2018/04/another-one.jpg )

In [21]:
 s = get_data_by_card_and_device_hash(train, 'b4f15ed9e7c1e0a', 'b62aa9813bf1ea8')[[ 'TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'device_hash','card_hash', 'V307']]

In [22]:
#magic
s['V307_diff'] = s['V307'].diff().shift(-1)
s['difference'] = s['V307_diff'] - s['TransactionAmt']

In [23]:
s

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,device_hash,card_hash,V307,V307_diff,difference
434965,3421965,1,11008695,51.281250,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,60.419998,51.274300,-0.006950
434966,3421966,1,11008754,51.281250,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,111.694298,51.274300,-0.006950
434968,3421968,1,11009166,37.875000,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,162.968597,37.871704,-0.003296
440288,3427288,1,11152691,15.367188,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,200.840302,15.369995,0.002808
445633,3432633,1,11328911,27.968750,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,216.210297,27.973404,0.004654
445653,3432653,1,11329400,27.968750,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,244.183701,27.973404,0.004654
445877,3432877,1,11352577,14.757812,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,272.157104,14.755188,-0.002625
446038,3433038,1,11365487,59.031250,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,286.912292,59.020813,-0.010437
446041,3433041,1,11365523,59.031250,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,345.933105,59.020782,-0.010468
447804,3434804,1,11401166,17.093750,C,b62aa9813bf1ea8,b4f15ed9e7c1e0a,404.953888,17.091400,-0.002350
